In [12]:
from sagemaker import image_uris
image_uris.retrieve(framework='pytorch',region='ap-southeast-1',version='1.12.0',py_version='py38',image_scope='training', instance_type='ml.c5.9xlarge')

'763104351884.dkr.ecr.ap-southeast-1.amazonaws.com/pytorch-training:1.12.0-cpu-py38'

In [2]:
%%sh

# The name of our algorithm
algorithm_name=pytorch-mer

cd container

account=$(aws sts get-caller-identity --query Account --output text)

# Get the region defined in the current configuration (default to us-west-2 if none defined)
region=$(aws configure get region)
region=${region:-ap-southeast-1}

fullname="${account}.dkr.ecr.${region}.amazonaws.com/${algorithm_name}:latest"

# If the repository doesn't exist in ECR, create it.

aws ecr describe-repositories --repository-names "${algorithm_name}" > /dev/null 2>&1

if [ $? -ne 0 ]
then
    aws ecr create-repository --repository-name "${algorithm_name}" > /dev/null
fi

# Get the login command from ECR and execute it directly
$(aws ecr get-login --region ap-southeast-1 --no-include-email)

# # Get the login command from ECR in order to pull down the SageMaker PyTorch image
$(aws ecr get-login --registry-ids 763104351884 --region ap-southeast-1 --no-include-email)

# Build the docker image locally with the image name and then push it to ECR
# with the full name.


docker build  -t ${algorithm_name} . --build-arg REGION=${region}
docker tag ${algorithm_name} ${fullname}

docker tag pytorch-mer:latest 695137441790.dkr.ecr.ap-southeast-1.amazonaws.com/pytorch-mer:latest

docker push 695137441790.dkr.ecr.ap-southeast-1.amazonaws.com/pytorch-mer:latest
# docker push ${fullname}

Login Succeeded
Login Succeeded
Sending build context to Docker daemon  110.6kB

Step 1/8 : ARG REGION=ap-southeast-1
Step 2/8 : FROM 763104351884.dkr.ecr.ap-southeast-1.amazonaws.com/pytorch-training:1.12.0-cpu-py38
 ---> 245b5fc43f49
Step 3/8 : RUN pip3 install openpyxl
 ---> Using cache
 ---> 5d7d9782f80f
Step 4/8 : RUN pip3 install timm
 ---> Using cache
 ---> 6952043511bc
Step 5/8 : ENV PATH="/opt/ml/code:${PATH}"
 ---> Using cache
 ---> ea070faaa420
Step 6/8 : COPY /codes /opt/ml/code
 ---> 0bad06265fc6
Step 7/8 : ENV SAGEMAKER_SUBMIT_DIRECTORY /opt/ml/code
 ---> Running in 393017dafa77
Removing intermediate container 393017dafa77
 ---> c0e57e1cb6d3
Step 8/8 : ENV SAGEMAKER_PROGRAM script.py
 ---> Running in 16877ead20b8
Removing intermediate container 16877ead20b8
 ---> b493701307d3
Successfully built b493701307d3
Successfully tagged pytorch-mer:latest
The push refers to repository [695137441790.dkr.ecr.ap-southeast-1.amazonaws.com/pytorch-mer]
d9ba2c99a70d: Preparing
70a9019a11

WARNING! Using --password via the CLI is insecure. Use --password-stdin.
WARNING! Your password will be stored unencrypted in /home/ec2-user/.docker/config.json.
Configure a credential helper to remove this warning. See
https://docs.docker.com/engine/reference/commandline/login/#credentials-store

WARNING! Using --password via the CLI is insecure. Use --password-stdin.
WARNING! Your password will be stored unencrypted in /home/ec2-user/.docker/config.json.
Configure a credential helper to remove this warning. See
https://docs.docker.com/engine/reference/commandline/login/#credentials-store

